Generate an intermediate csv file for the objects we are trying to identify

In [ ]:
from tools import get_data_file
from glob import glob
import pandas as pd
from PIL import Image
import os
from sklearn.model_selection import train_test_split

## Load and select classes

In [ ]:
class_file = get_data_file('raw', 'annotations/classes.txt')
annotation_files = set(glob(get_data_file('raw','annotations/*.txt')))
annotation_files.remove(class_file)

In [ ]:
with open(class_file) as readable:
    classes = { i:tag.strip() for i, tag in enumerate(readable) }
    rev_class = {v:k for k,v in classes.items()}

In [ ]:
classes_to_keep = set(["valentina", "valentina-negra", "botaneraaa"])
ids_to_keep = set([ rev_class[k] for k in classes_to_keep ])
print(ids_to_keep)

In [ ]:
classes

## Load image data

In [ ]:
def from_yolo_to_values(x1, y1, width, height, shape):
    img_width, img_height = shape
    x_rect = (x1 - width/2) * img_width
    y_rect = (y1 - height/2) * img_height
    width_rect = width * img_width
    height_rect = height * img_height
    return int(x_rect), int(y_rect), int(width_rect), int(height_rect)

In [ ]:
def get_image_path(image):
    image = get_data_file('raw', f'images/{image}')
    jpg = image + ".jpg"
    if os.path.exists(jpg):
        return jpg
    jpeg = image + ".jpeg"
    if os.path.exists(jpeg):
        return jpeg
    png = image + ".png"
    if os.path.exists(png):
        return png
    raise ValueError(f'No file for {image}')

In [ ]:
in_collection = []
for annotation_file in annotation_files:
    base_path = os.path.basename(annotation_file)
    name, extension = os.path.splitext(base_path)
    with open(annotation_file) as r:
        full_image_path =  get_image_path(name)
        im = Image.open(full_image_path)
        in_file = []
        for line in r:
            values = []
            string_values = line.split()
            class_id = int(string_values[0])
            if class_id not in ids_to_keep:
                continue
            values.append(full_image_path)
            x,y,w,h = from_yolo_to_values(*[float(f) for f in string_values[1:]], im.size)
            values.extend([ w, h ])
            values.append( classes[class_id])
            values.extend([ x, y, x+w, y+h ])
            in_file.append(values)
    in_collection.extend(in_file)

In [ ]:
labels = pd.DataFrame(in_collection, columns=['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])
labels.head()

In [ ]:
train, test = train_test_split(labels, test_size=0.2)

In [ ]:
len(train), len(test), type(train)

In [ ]:
train.to_csv(get_data_file('interim', 'train.csv'))
test.to_csv(get_data_file('interim', 'test.csv'))

Now, it is possible to use [Swaini's script](https://raw.githubusercontent.com/Swaini/object_detection_retraining/master/generate_tfrecord.py), located in `src/external/generate_tfrecord.py`, usage, from the root of this repo:

**Do not forget to edit the file `generate_tfrecord.py` to add all your classes**

```
PYTHONPATH=src python src/external/generate_tfrecord.py --input_csv=data/interim/train.csv  --output_tfrecord=data/interim/train.record

PYTHONPATH=src python src/external/generate_tfrecord.py --input_csv=data/interim/test.csv  --output_tfrecord=data/interim/test.record
```